* Lift scores - positive and negative attributes
* Topic Modeling
* Sentiment
* Cosine Similarity with mission statement
* Conditional Random Fields

* Rating vs. word length
* Recommendation by department, location
* Titles - Most frequent words in title review
* Most frequent words in highly rated reviews
* Most frequent words in low rated reviews
* Plot sentiments with time, anything stands out
* Plot sentiments across job titles, location - US Map of sentiments

Sentiment, topics, word length, frequent words across:
* High / low ratings - reviews
* CEO
* Outlook
* Recommends
* Title - for high/low ratings
* Job function
* Location
* Current / Former employee

In [2]:
import statsmodels.api as sm
import pandas as pd
import re,string
import nltk
from patsy import dmatrices
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import warnings
%pylab inline
import numpy as np
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

Populating the interactive namespace from numpy and matplotlib


In [19]:
df = pd.read_csv("Scraper/amazon_reviews.csv",encoding='ascii')
print(df.dtypes)
print(df.shape)
df.head()

date                object
title               object
overall_rating      object
employee_details    object
pros                object
cons                object
ratings_breakup     object
outlook             object
dtype: object
(9989, 8)


date                                              title  \
0  b'Sep 20, 2018'                      b'"An Amazing Place to Work"'   
1  b'Jan 10, 2016'                         b'"A great place to work"'   
2  b'Feb 20, 2016'                       b'"You Get What You Put In"'   
3  b'Dec 17, 2013'                b'"Exciting Work, Abusive Culture"'   
4  b'Mar 23, 2017'  b'"Can be amazing for some people, horrible fo...   

  overall_rating                                   employee_details  \
0         b'3.8'  b'Current Employee - Software Development Mana...   
1         b'5.0'     b'Former Employee - Transportation Specialist'   
2         b'5.0'           b'Current Employee - Anonymous Employee'   
3         b'5.0'   b'Current Employee - Senior Engineering Manager'   
4         b'3.0'  b'Current Employee - Software Development Mana...   

                                                pros  \
0  b"I've been at Amazon for a month now, and I'v...   
1  b'Great co-workers, friendly atmosphere, Lots ...   
2  b'Really smart people, a lot of opportunity fo...   
3  b'Jeff Bezos and his "S-Team" are brilliant an...   
4  b"Amazon is doing lot's of cool stuff...but lo...   

                                                cons  \
0  b"No cons, so far - seriously. Like I said, I'...   
1  b'Only a 30min lunch and a long drive from whe...   
2  b"You have to be self motivated. NO ONE will h...   
3  b'The management process is abusive, and I\'m ...   
4  b"- You're responsible for your own career pro...   

                                     ratings_breakup  \
0  [('Work/Life Balance', '4.0'), ('Culture &amp;...   
1  [('Work/Life Balance', '3.0'), ('Culture &amp;...   
2  [('Work/Life Balance', '4.0'), ('Culture &amp;...   
3  [('Work/Life Balance', '3.0'), ('Culture &amp;...   
4  [('Work/Life Balance', '3.0'), ('Culture &amp;...   

                                             outlook  
0  ['Recommends', 'Positive Outlook', 'Approves o...  
1  ['Recommends', 'Positive Outlook', 'Approves o...  
2  ['Recommends', 'Positive Outlook', 'Approves o...  
3  ["Doesn't Recommend", 'Positive Outlook', 'App...  
4  ['Recommends', 'Positive Outlook', 'Approves o...

In [191]:
# function to clean encoding from columns
def clean_encoding(s):
    s = s.astype(str)
    #print(type(s))
    s = s.map(lambda x: re.sub(r'^b[\'\"\'"\"''\s+]', '',x.lower()))
    s= s.map(lambda x: re.sub(r'[\'\"\'"\"''\s+]$', '',x.lower()))
    s = s.map(lambda x: re.sub(r'[^\w\s,.-]', '',x.lower()))
    return(s)

df2 = df.apply(clean_encoding, axis=0)
df2.head()

date                                              title  \
0  sep 20, 2018                           an amazing place to work   
1  jan 10, 2016                              a great place to work   
2  feb 20, 2016                            you get what you put in   
3  dec 17, 2013                     exciting work, abusive culture   
4  mar 23, 2017  can be amazing for some people, horrible for o...   

  overall_rating                                 employee_details  \
0            3.8  current employee - software development manager   
1            5.0      former employee - transportation specialist   
2            5.0            current employee - anonymous employee   
3            5.0    current employee - senior engineering manager   
4            3.0  current employee - software development manager   

                                                pros  \
0  ive been at amazon for a month now, and ive se...   
1  great co-workers, friendly atmosphere, lots of...   
2  really smart people, a lot of opportunity for ...   
3  jeff bezos and his s-team are brilliant and co...   
4  amazon is doing lots of cool stuff...but lots ...   

                                                cons  \
0  no cons, so far - seriously. like i said, im s...   
1  only a 30min lunch and a long drive from where...   
2  you have to be self motivated. no one will hol...   
3  the management process is abusive, and im curr...   
4  - youre responsible for your own career progre...   

                                     ratings_breakup  \
0  worklife balance, 4.0, culture amp values, 5.0...   
1  worklife balance, 3.0, culture amp values, 5.0...   
2  worklife balance, 4.0, culture amp values, 5.0...   
3  worklife balance, 3.0, culture amp values, 3.0...   
4  worklife balance, 3.0, culture amp values, 3.0...   

                                             outlook  
0  recommends, positive outlook, approves of ceo,...  
1  recommends, positive outlook, approves of ceo,...  
2  recommends, positive outlook, approves of ceo,...  
3  doesnt recommend, positive outlook, approves o...  
4  recommends, positive outlook, approves of ceo,...

## Cleaning dataset
Breaking out employee details, ratings and outlook

In [192]:
df2['employee_type'],df2['designation'] = df2['employee_details'].str.split('-', 1).str

In [193]:
x1= []
x2 = []
x3 = []
x4 = []
x5 = []
def clean_ratings(s):
    x = s.split(",")
    x1.append(float(x[1]))
    x2.append(float(x[3]))
    x3.append(float(x[5]))
    x4.append(float(x[7]))
    x5.append(float(x[9]))

df2['ratings_breakup'].map(clean_ratings)
df2['r_work_life'] = x1
df2['r_culture'] = x2
df2['r_career'] = x3
df2['r_comp'] = x4
df2['r_management'] = x5

In [199]:
x1= []
x2 = []
x3 = []
def clean_outlook(s):
    x = s.split(",")
    x1.append(x[0])
    x2.append(x[1])
    x3.append(x[2])
    
df2['outlook'].map(clean_outlook)
df2['company_reco'] = x1
df2['company_outlook'] = x2
df2['ceo'] = x3
print(df2.dtypes)
df2.head()

date                 object
title                object
overall_rating       object
employee_details     object
pros                 object
cons                 object
ratings_breakup      object
outlook              object
employee_type        object
designation          object
r_work_life         float64
r_culture           float64
r_career            float64
r_comp              float64
r_management        float64
company_reco         object
company_outlook      object
ceo                  object
dtype: object


date                                              title  \
0  sep 20, 2018                           an amazing place to work   
1  jan 10, 2016                              a great place to work   
2  feb 20, 2016                            you get what you put in   
3  dec 17, 2013                     exciting work, abusive culture   
4  mar 23, 2017  can be amazing for some people, horrible for o...   

  overall_rating                                 employee_details  \
0            3.8  current employee - software development manager   
1            5.0      former employee - transportation specialist   
2            5.0            current employee - anonymous employee   
3            5.0    current employee - senior engineering manager   
4            3.0  current employee - software development manager   

                                                pros  \
0  ive been at amazon for a month now, and ive se...   
1  great co-workers, friendly atmosphere, lots of...   
2  really smart people, a lot of opportunity for ...   
3  jeff bezos and his s-team are brilliant and co...   
4  amazon is doing lots of cool stuff...but lots ...   

                                                cons  \
0  no cons, so far - seriously. like i said, im s...   
1  only a 30min lunch and a long drive from where...   
2  you have to be self motivated. no one will hol...   
3  the management process is abusive, and im curr...   
4  - youre responsible for your own career progre...   

                                     ratings_breakup  \
0  worklife balance, 4.0, culture amp values, 5.0...   
1  worklife balance, 3.0, culture amp values, 5.0...   
2  worklife balance, 4.0, culture amp values, 5.0...   
3  worklife balance, 3.0, culture amp values, 3.0...   
4  worklife balance, 3.0, culture amp values, 3.0...   

                                             outlook      employee_type  \
0  recommends, positive outlook, approves of ceo,...  current employee    
1  recommends, positive outlook, approves of ceo,...   former employee    
2  recommends, positive outlook, approves of ceo,...  current employee    
3  doesnt recommend, positive outlook, approves o...  current employee    
4  recommends, positive outlook, approves of ceo,...  current employee    

                     designation  r_work_life  r_culture  r_career  r_comp  \
0   software development manager          4.0        5.0       5.0     5.0   
1      transportation specialist          3.0        5.0       4.0     3.0   
2             anonymous employee          4.0        5.0       5.0     5.0   
3     senior engineering manager          3.0        3.0       3.0     4.0   
4   software development manager          3.0        3.0       5.0     5.0   

   r_management      company_reco    company_outlook               ceo  
0           5.0        recommends   positive outlook   approves of ceo  
1           5.0        recommends   positive outlook   approves of ceo  
2           5.0        recommends   positive outlook   approves of ceo  
3           4.0  doesnt recommend   positive outlook   approves of ceo  
4           4.0        recommends   positive outlook   approves of ceo

In [200]:
df2.overall_rating= df2.overall_rating.astype(float)
df2.describe()

overall_rating  r_work_life    r_culture     r_career       r_comp  \
count     9989.000000  9989.000000  9989.000000  9989.000000  9989.000000   
mean         3.573711     2.899990     3.699970     4.000000     4.399940   
std          1.294021     0.943492     1.187549     1.183334     0.800058   
min          1.000000     1.000000     2.000000     1.000000     3.000000   
25%          3.000000     2.000000     3.000000     4.000000     4.000000   
50%          4.000000     3.000000     3.000000     4.000000     5.000000   
75%          5.000000     4.000000     5.000000     5.000000     5.000000   
max          5.000000     4.000000     5.000000     5.000000     5.000000   

       r_management  
count   9989.000000  
mean       4.000100  
std        1.264998  
min        1.000000  
25%        3.000000  
50%        5.000000  
75%        5.000000  
max        5.000000

In [204]:
print(df2.employee_type.value_counts())
df2.groupby('employee_type')[['overall_rating']].mean()

current employee     6509
former employee      3480
Name: employee_type, dtype: int64


overall_rating
employee_type                    
current employee         3.840498
former employee          3.074713

In [219]:
print(df2.designation.value_counts()[:20])
top_designations = df2.designation.value_counts().index[:20]
df2[df2.designation.isin(top_designations)].groupby('designation')[['overall_rating']].mean().plot(kind='bar')
#Lowest rating by pickers and packers

 anonymous employee                 4472
 warehouse associate                 355
 fulfillment associate               345
 software development engineer       137
 associate                           123
 area manager                        121
 picker                               96
 sortation associate                  96
 stower                               90
 process assistant                    85
 packer                               67
 order picker                         67
 program manager                      62
 fc associate                         61
 customer service representative      60
 software engineer                    54
 operations manager                   51
 software development manager         49
 inbound stow                         49
 senior product manager               48
Name: designation, dtype: int64


overall_rating
designation                                     
 anonymous employee                     3.585197
 area manager                           3.322314
 associate                              3.040650
 customer service representative        3.550000
 fc associate                           3.196721
 fulfillment associate                  3.228986
 inbound stow                           3.244898
 operations manager                     3.666667
 order picker                           2.940299
 packer                                 3.402985
 picker                                 2.635417
 process assistant                      3.552941
 program manager                        4.096774
 senior product manager                 4.062500
 software development engineer          3.941606
 software development manager           4.281633
 software engineer                      3.333333
 sortation associate                    3.156250
 stower                                 2.811111
 warehouse associate                    3.090141

In [220]:
print(df2.company_reco.value_counts())
df2.groupby('company_reco')[['overall_rating']].mean()

recommends          7991
doesnt recommend    1998
Name: company_reco, dtype: int64


overall_rating
company_reco                    
doesnt recommend        3.571572
recommends              3.574246

In [221]:
print(df2.ceo.value_counts())
df2.groupby('ceo')[['overall_rating']].mean()

 approves of ceo       6992
 no opinion of ceo     1998
 disapproves of ceo     999
Name: ceo, dtype: int64


overall_rating
ceo                                
 approves of ceo           3.551030
 disapproves of ceo        3.594595
 no opinion of ceo         3.642643

## Tokenizing and removing stop words

In [245]:
stop = set(stopwords.words('english'))
punc = string.punctuation    

def clean_tokenize(s):
    s = re.sub(r'[^\w\s]', '',s.lower())
    return([word for word in word_tokenize(s) if word not in stop if word not in punc])

df2['pros_clean'] = df2.pros.map(clean_tokenize)
df2['cons_clean'] = df2.cons.map(clean_tokenize)
df2['title_clean'] = df2.title.map(clean_tokenize)
# df2['pros_clean'] = df2.pros.apply(lambda x: [word for word in word_tokenize(x) if word not in stop if word not in punc])
# df2['cons_clean'] = df2.cons.apply(lambda x: [word for word in word_tokenize(x) if word not in stop if word not in punc])
# df2['title_clean'] = df2.title.apply(lambda x: [word for word in word_tokenize(x) if word not in stop if word not in punc])

In [246]:
df2["pros_cunique"] = df2.pros_clean.map(lambda x: sorted(set(x)))
df2["cons_cunique"] = df2.cons_clean.map(lambda x: sorted(set(x)))
df2["title_cunique"] = df2.title_clean.map(lambda x: sorted(set(x)))
df2.head()

date                                              title  \
0  sep 20, 2018                           an amazing place to work   
1  jan 10, 2016                              a great place to work   
2  feb 20, 2016                            you get what you put in   
3  dec 17, 2013                     exciting work, abusive culture   
4  mar 23, 2017  can be amazing for some people, horrible for o...   

   overall_rating                                 employee_details  \
0             3.8  current employee - software development manager   
1             5.0      former employee - transportation specialist   
2             5.0            current employee - anonymous employee   
3             5.0    current employee - senior engineering manager   
4             3.0  current employee - software development manager   

                                                pros  \
0  ive been at amazon for a month now, and ive se...   
1  great co-workers, friendly atmosphere, lots of...   
2  really smart people, a lot of opportunity for ...   
3  jeff bezos and his s-team are brilliant and co...   
4  amazon is doing lots of cool stuff...but lots ...   

                                                cons  \
0  no cons, so far - seriously. like i said, im s...   
1  only a 30min lunch and a long drive from where...   
2  you have to be self motivated. no one will hol...   
3  the management process is abusive, and im curr...   
4  - youre responsible for your own career progre...   

                                     ratings_breakup  \
0  worklife balance, 4.0, culture amp values, 5.0...   
1  worklife balance, 3.0, culture amp values, 5.0...   
2  worklife balance, 4.0, culture amp values, 5.0...   
3  worklife balance, 3.0, culture amp values, 3.0...   
4  worklife balance, 3.0, culture amp values, 3.0...   

                                             outlook      employee_type  \
0  recommends, positive outlook, approves of ceo,...  current employee    
1  recommends, positive outlook, approves of ceo,...   former employee    
2  recommends, positive outlook, approves of ceo,...  current employee    
3  doesnt recommend, positive outlook, approves o...  current employee    
4  recommends, positive outlook, approves of ceo,...  current employee    

                     designation                 ...                   \
0   software development manager                 ...                    
1      transportation specialist                 ...                    
2             anonymous employee                 ...                    
3     senior engineering manager                 ...                    
4   software development manager                 ...                    

   r_management      company_reco    company_outlook               ceo  \
0           5.0        recommends   positive outlook   approves of ceo   
1           5.0        recommends   positive outlook   approves of ceo   
2           5.0        recommends   positive outlook   approves of ceo   
3           4.0  doesnt recommend   positive outlook   approves of ceo   
4           4.0        recommends   positive outlook   approves of ceo   

                                          pros_clean  \
0  [ive, amazon, month, ive, seen, none, horror, ...   
1  [great, coworkers, friendly, atmosphere, lots,...   
2  [really, smart, people, lot, opportunity, grow...   
3  [jeff, bezos, steam, brilliant, continue, make...   
4  [amazon, lots, cool, stuffbut, lots, boring, s...   

                                          cons_clean  \
0  [cons, far, seriously, like, said, im, sure, a...   
1                  in, lunch, long, drive, live]   
2  [self, motivated, one, hold, hand, tell, youre...   
3  [management, process, abusive, im, currently, ...   
4  [youre, responsible, career, progression, find...   

                                        pros_cunique  \
0  [although, amazon, bad, collaborate, communica...   
1  [advancement, atmosphere, coworkers, f

## Top words for pros, cons, title

In [247]:
#This step takes time to run
pros = df2['pros_cunique'].sum()
cons = df2['cons_cunique'].sum()
title = df2['title_cunique'].sum()

In [248]:
pros_freq = nltk.FreqDist(pros)
print("*****top words: pros********")
print(pros_freq.most_common(20))
cons_freq = nltk.FreqDist(cons)
print("*****top words: cons********")
print(cons_freq.most_common(20))
title_freq = nltk.FreqDist(title)
print("*****top words: title********")
print(title_freq.most_common(20))

*****top words: pros********
[('work', 2960), ('great', 2396), ('good', 2098), ('benefits', 2009), ('pay', 1751), ('people', 1446), ('company', 1087), ('amazon', 953), ('time', 907), ('get', 775), ('environment', 740), ('opportunities', 729), ('job', 726), ('hours', 679), ('working', 640), ('fast', 589), ('easy', 572), ('day', 568), ('lot', 562), ('learn', 525)]
*****top words: cons********
[('work', 2507), ('hours', 1294), ('get', 1163), ('management', 1081), ('time', 1048), ('long', 1004), ('xe2x80xa6', 897), ('people', 878), ('hard', 753), ('like', 728), ('amazon', 705), ('company', 700), ('dont', 697), ('job', 687), ('pay', 671), ('employees', 647), ('nshow', 629), ('lot', 581), ('working', 572), ('balance', 567)]
*****top words: title********
[('work', 1123), ('great', 999), ('amazon', 989), ('associate', 867), ('place', 730), ('good', 688), ('company', 615), ('manager', 572), ('job', 482), ('warehouse', 378), ('fulfillment', 307), ('engineer', 226), ('software', 199), ('experienc

## Top words for pros, cons, title (below mean rating)

In [249]:
#checking if words vary for over mean and below mean rating of 3.5
df_low_rating = df2[df2.overall_rating < 3.5]
pros_lr = df_low_rating['pros_cunique'].sum()
cons_lr = df_low_rating['cons_cunique'].sum()
title_lr = df_low_rating['title_cunique'].sum()

In [253]:
pros_freq = nltk.FreqDist(pros_lr)
print("*****top words: pros********")
print(pros_freq.most_common(20))
cons_freq = nltk.FreqDist(cons_lr)
print("*****top words: cons********")
print(cons_freq.most_common(20))
title_freq = nltk.FreqDist(title_lr)
print("*****top words: title********")
print(title_freq.most_common(20))

*****top words: pros********
[('good', 997), ('benefits', 988), ('work', 955), ('pay', 925), ('great', 753), ('people', 512), ('time', 450), ('get', 370), ('job', 355), ('company', 338), ('hours', 327), ('decent', 289), ('amazon', 285), ('day', 268), ('easy', 262), ('working', 210), ('stock', 202), ('lot', 202), ('lots', 198), ('nice', 190)]
*****top words: cons********
[('work', 1247), ('management', 759), ('xe2x80xa6', 669), ('hours', 604), ('get', 600), ('people', 497), ('time', 487), ('nshow', 465), ('employees', 435), ('long', 430), ('pay', 422), ('like', 418), ('job', 372), ('dont', 361), ('managers', 341), ('amazon', 324), ('hard', 317), ('company', 304), ('balance', 270), ('working', 265)]
*****top words: title********
[('amazon', 411), ('associate', 396), ('work', 356), ('good', 237), ('job', 215), ('warehouse', 202), ('manager', 195), ('great', 163), ('fulfillment', 151), ('company', 144), ('place', 135), ('bad', 103), ('pay', 94), ('dont', 88), ('hard', 79), ('customer', 79)

## Top words for pros, cons, title (above mean rating)

In [251]:
#checking if words vary for over mean and below mean rating of 3.5
df_high_rating = df2[df2.overall_rating >= 3.5]
pros_hr = df_high_rating['pros_cunique'].sum()
cons_hr = df_high_rating['cons_cunique'].sum()
title_hr = df_high_rating['title_cunique'].sum()

In [252]:
pros_freq = nltk.FreqDist(pros_hr)
print("*****top words: pros********")
print(pros_freq.most_common(20))
cons_freq = nltk.FreqDist(cons_hr)
print("*****top words: cons********")
print(cons_freq.most_common(20))
title_freq = nltk.FreqDist(title_hr)
print("*****top words: title********")
print(title_freq.most_common(20))

*****top words: pros********
[('work', 2005), ('great', 1643), ('good', 1101), ('benefits', 1021), ('people', 934), ('pay', 826), ('company', 749), ('amazon', 668), ('environment', 586), ('opportunities', 551), ('time', 457), ('fast', 435), ('working', 430), ('get', 405), ('place', 400), ('team', 382), ('culture', 373), ('job', 371), ('smart', 370), ('lot', 360)]
*****top words: cons********
[('work', 1260), ('hours', 690), ('long', 574), ('get', 563), ('time', 561), ('hard', 436), ('company', 396), ('amazon', 381), ('people', 381), ('sometimes', 381), ('lot', 346), ('dont', 336), ('management', 322), ('job', 315), ('like', 310), ('working', 307), ('balance', 297), ('much', 289), ('need', 267), ('many', 256)]
*****top words: title********
[('great', 836), ('work', 767), ('place', 595), ('amazon', 578), ('associate', 471), ('company', 471), ('good', 451), ('manager', 377), ('job', 267), ('warehouse', 176), ('engineer', 166), ('fulfillment', 156), ('software', 143), ('experience', 127), 

## Word Length for pros, cons, title

## Bigrams and Trigrams for pros, cons, title

In [ ]:
#Stemming before calculating bigrams and trigrams

In [258]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [259]:
def bigram_maker(tokenizedList,num):
    bg = BigramCollocationFinder.from_words(tokenizedList)
    bg.apply_freq_filter(10)
    bg_measures = bg.score_ngrams(bigram_measures.pmi)
    bg_sorted = sorted(bg_measures, key = lambda x: x[1])
    bg_sorted_20 = bg_sorted[-num:][::-1]
    print("Top 20 bigrams: \n")
    print(bg_sorted_20)

In [260]:
def trigram_maker(tokenizedList,num):
    tg = TrigramCollocationFinder.from_words(tokenizedList)
    tg.apply_freq_filter(10)
    tg_measures = tg.score_ngrams(trigram_measures.pmi)
    tg_sorted = sorted(tg_measures, key = lambda x: x[1])
    tg_sorted_20 = tg_sorted[-num:][::-1]
    print("Top 20 Trigrams: \n")
    print(tg_sorted_20)

In [262]:
print("******pros******")
bigram_maker(pros,20)
print("******cons******")
bigram_maker(cons,20)

******pros******
Top 20 bigrams: 

[(('cutting', 'edge'), 10.126790100349575), (('code', 'dress'), 8.533971066434573), (('pto', 'upt'), 8.273854143983074), (('want', 'whenever'), 7.954583436181343), (('peak', 'season'), 7.919044454977527), (('unpaid', 'vacation'), 7.776958362051845), (('discount', 'employee'), 7.758820854727512), (('data', 'driven'), 7.742410496995454), (('switch', 'teams'), 7.730026772539622), (('food', 'free'), 7.647359114869786), (('huge', 'impact'), 7.6216326396971965), (('shifts', 'short'), 7.385668876692057), (('career', 'choice'), 7.330437672349442), (('leadership', 'lives'), 7.234069277877209), (('shares', 'stock'), 7.1657156122579), (('unpaid', 'use'), 7.139470255345243), (('health', 'insurance'), 6.910763437241497), (('3', '4'), 6.902457372831412), (('youll', 'youre'), 6.872432151425116), (('10', '4'), 6.864463207845372)]
******cons******
Top 20 bigrams: 

[(('red', 'tape'), 11.63940021472747), (('mind', 'numbing'), 10.948015654870158), (('hand', 'hold'), 9.4

In [261]:
print("******pros******")
trigram_maker(pros,20)
print("******cons******")
trigram_maker(cons,20)

******pros******
Top 20 Trigrams: 

[(('time', 'unpaid', 'vacation'), 13.753303807572347), (('benefits', 'career', 'choice'), 12.124376208709258), (('working', 'xe2x80xa6', 'years'), 11.198004784293193), (('worked', 'working', 'xe2x80xa6'), 11.076531370362211), (('good', 'health', 'insurance'), 10.769714909477845), (('meet', 'new', 'people'), 10.174474848558283), (('great', 'health', 'insurance'), 9.993139178545174), (('people', 'projects', 'smart'), 9.80616040024232), (('work', 'worked', 'xe2x80xa6'), 9.5451499098459), (('benefits', 'day', 'first'), 9.452400298544806), (('1', 'benefits', 'day'), 9.350106652368673), (('work', 'xe2x80xa6', 'year'), 9.29561134649633), (('4', 'benefits', 'day'), 9.07416476275549), (('people', 'really', 'smart'), 9.014125750943492), (('work', 'xe2x80xa6', 'years'), 8.988551418664244), (('learn', 'lot', 'opportunities'), 8.960123520139202), (('get', 'go', 'good'), 8.920184153251345), (('work', 'working', 'xe2x80xa6'), 8.83632060916775), (('work', 'worked', 

## Bigrams and Trigrams with overall rating, employee type, designation, other fields

## Stemming and Topic Modeling